In [1]:
#import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

In [3]:
hotels_list=[ ]

#since the first 30 enaatries on the first page has a different url, we make a different function for it (there could be so many ways to do but ....)

def get_first_30_hotels():
    #url to extract the first 30 hotels of Nepal
    url="https://www.tripadvisor.com/Hotels-g293890-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region-Hotels.html"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    hotels = soup.find_all('div', {'class':'ui_column is-8 main_col allowEllipsis'})
    for item in hotels:
        #title
        title=item.find('a',{'class':'property_title prominent'}).text.strip()
        
        #average rating
        try:           
            #avg rating out of 5
            avg_rating=item.find('a',{'data-clicksource':"BubbleRating"})["alt"].split(" ")[0] 
        except:
            avg_rating='Nan'
        
        #price 
        price=item.find('div',{'data-clickpart':"chevron_price"}).text.strip()
        
        #hotel merchandise message 
        try:
            hotel_merchandise_msg=item.find('div',{'class':"prw_rup prw_hotels_merchandise_messages"}).text
        except:
            hotel_merchandise_msg='Nan'
                
        #no of votes
        try:
            voted_by=item.find('a',{'class':'review_count'}).text.strip().replace(',','').split(" ")[0]
        except:
            voted_by='0'
        
        #services
        try:
            services=item.find('div',{'class':'prw_rup prw_common_hotel_icons_list linespace is-shown-at-tablet'}).text
        except:
            services='Nan'
        

        hotel = {
        'title':title,
        'avg_rating':avg_rating,
        'price':price,
        'voted_by':int(voted_by),
        'services':services,
        'hotel_merchandise_msg':hotel_merchandise_msg
        }        
        hotels_list.append(hotel)
    return

In [4]:
get_first_30_hotels()

In [5]:
#for the remaining pages 
def get_remaining_hotels(page):
    #url to extract the first 30 hotels of Nepal
    url=f"https://www.tripadvisor.com/Hotels-g293890-oa{page}-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region-Hotels.html"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    hotels = soup.find_all('div', {'class':'ui_column is-8 main_col allowEllipsis'})
    for item in hotels:
        #title
        title=item.find('a',{'class':'property_title prominent'}).text.strip()
        
        #average rating
        try:           
            #avg rating out of 5
            avg_rating=item.find('a',{'data-clicksource':"BubbleRating"})["alt"].split(" ")[0] 
        except:
            avg_rating='Nan'
        
        #price 
        try:
            price=item.find('div',{'data-clickpart':"chevron_price"}).text.strip()
        except:
            price='Nan'
        #hotel merchandise message 
        try:
            hotel_merchandise_msg=item.find('div',{'class':"prw_rup prw_hotels_merchandise_messages"}).text
        except:
            hotel_merchandise_msg='Nan'
                
        #no of votes
        try:
            voted_by=item.find('a',{'class':'review_count'}).text.strip().replace(',','').split(" ")[0]
        except:
            voted_by='0'
        
        #services
        try:
            services=item.find('div',{'class':'prw_rup prw_common_hotel_icons_list linespace is-shown-at-tablet'}).text
        except:
            services='Nan'
        

        hotel = {
        'title':title,
        'avg_rating':avg_rating,
        'price':price,
        'voted_by':int(voted_by),
        'services':services,
        'hotel_merchandise_msg':hotel_merchandise_msg,
        }        
        hotels_list.append(hotel)
    return

In [6]:
#for the remaining 3000+ attractions
page=30 #initializing page number 

while page<=1500:
    get_remaining_hotels(page)
    print(f"done #{page} hotels")
    page+=30 #keep increasing by 30 in each iteration

done #30 hotels
done #60 hotels


ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read, 10240 more expected)', IncompleteRead(0 bytes read, 10240 more expected))

In [ ]:
#store in a Pandas dataframe 
df = pd.DataFrame(hotels_list)
print(df)

In [ ]:
#save to a csv file 
df.to_csv('hotels_ktm.csv', index=False)